## <font color= #13c113  >Word Embeddings</font> with Keras

<img src=https://images-na.ssl-images-amazon.com/images/I/41DWjHboiyL._SX258_BO1,204,203,200_.jpg height="240" width="200">

### Adapted from:

#### [6.1-using-word-embeddings](https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/6.1-using-word-embeddings.ipynb)

By François Chollet


<br>

* <font size=5 color='green'>[MSTC](http://mstc.ssr.upm.es/big-data-track) seminar on Deep Learning, Tensorflow & Keras</font>



---

## A popular and powerful way to associate a vector with a word is the use of dense "word vectors", also called "word embeddings".

<img src=https://s3.amazonaws.com/book.keras.io/img/ch6/word_embeddings.png height="300" width="400">
from François Chollet book.keras.io

- #### One-hot encoding are binary, sparse (mostly made of zeros) and very high-dimensional (same dimensionality as the number of words in the vocabulary), "word embeddings" are low-dimensional floating point vectors (i.e. "dense" vectors, as opposed to sparse vectors).
- #### Unlike one-hot encoding, word embeddings are learned from data.

It is common to see word embeddings that are 256-dimensional, 512-dimensional, or 1024-dimensional when dealing with very large vocabularies. On the other hand, one-hot encoding words generally leads to vectors that are 20,000-dimensional or higher (capturing a vocabulary of 20,000 token in this case). So, word embeddings pack more information into far fewer dimensions.



---

### There are two ways to obtain word embeddings:

-    Learn word embeddings jointly with the main task you care about (e.g. document classification or sentiment prediction). In this setup, you would start with random word vectors, then **learn your word vectors in the same way that you learn the weights of a neural network**.

<br>
-    Load into your model word embeddings that were pre-computed using a different machine learning task than the one you are trying to solve. These are called **"pre-trained word embeddings"**.



---

## Learning word embeddings with the $Embedding$ layer

- It is reasonable to learn a new embedding space with every new task.
- Thankfully, backpropagation makes this really easy,
- and Keras makes it even easier.

      It's just about learning the weights of a layer: the Embedding layer.

In [1]:
! pip install --upgrade keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages
Requirement already up-to-date: six>=1.9.0 in /usr/local/lib/python3.6/dist-packages (from keras)
    100% |████████████████████████████████| 256kB 2.5MB/s 
Requirement already up-to-date: numpy>=1.9.1 in /usr/local/lib/python3.6/dist-packages (from keras)
    100% |████████████████████████████████| 50.0MB 26kB/s 
  Running setup.py bdist_wheel for pyyaml ... - \ done
  Stored in directory: /content/.cache/pip/wheels/2c/f7/79/13f3a12cd723892437c0cfbde1230ab4d82947ff7b3839a4fc
Successfully built pyyaml
  Found existing installation: PyYAML 3.11
    Uninstalling PyYAML-3.11:
      Successfully uninstalled PyYAML-3.11
  Found existing installation: scipy 0.19.1
    Uninstalling scipy-0.19.1:
      Successfully uninstalled scipy-0.19.1


In [2]:
from keras.layers import Embedding

# The Embedding layer takes at least two arguments:
# the number of possible tokens, here 1000 (1 + maximum word index),
# and the dimensionality of the embeddings, here 64.
embedding_layer = Embedding(1000, 64)

Using TensorFlow backend.



---

### The Embedding layer is best understood as a dictionary:
- It takes as input integers, it looks up these integers into an internal dictionary, and it returns the associated vectors.

- The Embedding layer takes as input a 2D tensor of integers, of shape (samples, sequence_length), where each entry is a sequence of integers.

- The Embedding layer returns a 3D floating point tensor, of shape (samples, sequence_length, embedding_dimensionality).





---
### Training the Embedding layer:

- When you instantiate an Embedding layer, its weights (its internal dictionary of token vectors) are initially random, just like with any other layer.

- During training, these word vectors will be gradually adjusted via backpropagation, structuring the space into something that the downstream model can exploit. 

- Once fully trained, your embedding space will show a lot of structure -- a kind of structure specialized for the specific problem you were training your model for.


---


## See example using the IMDB movie review sentiment prediction.

    "IMDB dataset", a set of 50,000 highly-polarized reviews from the Internet Movie Database. They are split into 25,000 reviews for training and 25,000 reviews for testing, each set consisting in 50% negative and 50% positive reviews.

    Just like the MNIST dataset, the IMDB dataset comes packaged with Keras. It has already been preprocessed: the reviews (sequences of words) have been turned into sequences of integers, where each integer stands for a specific word in a dictionary.


Let's quickly prepare the data. 

- We will restrict the movie reviews to the top 10,000 most common words
- and cut the reviews after only 20 words. 

**Our network will simply learn 8-dimensional embeddings for each of the 10,000 words, turn the input integer sequences (2D integer tensor) into embedded sequences (3D float tensor), flatten the tensor to 2D, and train a single Dense layer on top for classification.**

In [3]:
from keras.datasets import imdb
from keras import preprocessing

# Number of words to consider as features
max_features = 10000
# Cut texts after this number of words 
# (among top max_features most common words)
maxlen = 20

# Load the data as lists of integers.
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# This turns our lists of integers
# into a 2D integer tensor of shape `(samples, maxlen)`
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

17465344/17464789 [==============================] - 1s 0us/step


In [4]:
print('Train data shape:', x_train.shape)
print('Test  data shape:', x_test.shape)




Train data shape: (25000, 20)
Test  data shape: (25000, 20)


---
## Model with a first layer Embeddings on the input sequence + Flatten + Dense

- Define the model
- Compile it
- Fit train data & evaluate with test data

In [5]:
from keras.models import Sequential
from keras.layers import Flatten, Dense

batch_size=32

model = Sequential()

#### NOTE that Embedding requires input_length #######################################
# We specify the maximum input length to our Embedding layer
# so we can later flatten the embedded inputs
# This argument is required if you are going to connect Flatten then Dense layers upstream

model.add(Embedding(10000, 8, input_length=maxlen))
# After the Embedding layer, 
# our activations have shape `(samples, maxlen, 8)`.

# We flatten the 3D tensor of embeddings 
# into a 2D tensor of shape `(samples, maxlen * 8)`
model.add(Flatten())

# We add the classifier on top
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=batch_size,
                    validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 2s 76us/step - loss: 0.6759 - acc: 0.6047 - val_loss: 0.6398 - val_acc: 0.6806
Epoch 2/10
20000/20000 [==============================] - 2s 75us/step - loss: 0.5658 - acc: 0.7429 - val_loss: 0.5467 - val_acc: 0.7208
Epoch 3/10
20000/20000 [==============================] - 2s 77us/step - loss: 

20000/20000 [==============================] - 1s 73us/step - loss: 0.4264 - acc: 0.8077 - val_loss: 0.5008 - val_acc: 0.7452
Epoch 5/10
20000/20000 [==============================] - 1s 72us/step - loss: 0.3931 - acc: 0.8254 - val_loss: 0.4981 - val_acc: 0.7544
Epoch 6/10
20000/20000 [==============================] - 1s 72us/step - loss: 0.3669 - acc: 0.8397 - val_loss: 0.5013 - val_acc: 0.7536
Epoch 7/10
20000/20000 [==============================] - 1s 74us/step - loss: 0.3436 - acc: 0.8534 - val_loss: 0.5051 - val_acc: 0.7520
Epoch 8/10
20000/20000 [==============================] - 1s 74us/step - loss: 0.3225 - acc: 0.8655 - val_loss: 0.5131 - val_acc: 0.7486
Epoch 9/10
20000/20000 [==============================] - 1s 74us/step - loss: 0.3024 - acc: 0.8763 - val_loss: 0.5212 - val_acc: 0.7490
Epoch 10/10
20000/20000 [==============================] - 1s 75us/step - loss: 0.2841 - acc: 0.8858 - val_loss: 0.5301 - val_acc: 0.7462


In [6]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score without RNN:', score)
print('Test accuracy without RNN:', acc)

25000/25000 [==============================] - 1s 25us/step
Test score without RNN: 0.5212544250965119
Test accuracy without RNN: 0.75596


---
## Now test a Model with a first layer Embeddings on every input + LSTM + Dense

- Define the model
- Compile it
- Fit train data & evaluate with test data

In [15]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, LSTM

batch_size=32

model = Sequential()

model.add(Embedding(10000, 8))
model.add(LSTM(8, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=batch_size,
                    validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 8)           80000     
_________________________________________________________________
lstm_3 (LSTM)                (None, 8)                 544       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 9         
Total params: 80,553
Trainable params: 80,553
Non-trainable params: 0
_________________________________________________________________
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 6624/20000 [========>.....................] - ETA: 20s - loss: 0.6845 - acc: 0.5791

20000/20000 [==============================] - 30s 2ms/step - loss: 0.6225 - acc: 0.6589 - val_loss: 0.5401 - val_acc: 0.7300
Epoch 2/10
20000/20000 [==============================] - 30s 1ms/step - loss: 0.4886 - acc: 0.7665 - val_loss: 0.5014 - val_acc: 0.7480
Epoch 3/10
 1216/20000 [>.............................] - ETA: 26s - loss: 0.4508 - acc: 0.8026

20000/20000 [==============================] - 29s 1ms/step - loss: 0.4475 - acc: 0.7945 - val_loss: 0.4979 - val_acc: 0.7498
Epoch 4/10
20000/20000 [==============================] - 29s 1ms/step - loss: 0.4262 - acc: 0.8030 - val_loss: 0.4928 - val_acc: 0.7556
Epoch 5/10
   96/20000 [..............................] - ETA: 26s - loss: 0.3415 - acc: 0.8542

20000/20000 [==============================] - 29s 1ms/step - loss: 0.4107 - acc: 0.8163 - val_loss: 0.4883 - val_acc: 0.7570
Epoch 6/10
19680/20000 [============================>.] - ETA: 0s - loss: 0.3969 - acc: 0.8220

20000/20000 [==============================] - 29s 1ms/step - loss: 0.3968 - acc: 0.8221 - val_loss: 0.4981 - val_acc: 0.7502
Epoch 7/10
20000/20000 [==============================] - 29s 1ms/step - loss: 0.3919 - acc: 0.8265 - val_loss: 0.4924 - val_acc: 0.7564
Epoch 8/10
 4448/20000 [=====>........................] - ETA: 21s - loss: 0.3794 - acc: 0.8269

20000/20000 [==============================] - 29s 1ms/step - loss: 0.3873 - acc: 0.8273 - val_loss: 0.5005 - val_acc: 0.7554
Epoch 9/10
20000/20000 [==============================] - 29s 1ms/step - loss: 0.3780 - acc: 0.8326 - val_loss: 0.4948 - val_acc: 0.7550
Epoch 10/10
  992/20000 [>.............................] - ETA: 26s - loss: 0.3941 - acc: 0.8276

20000/20000 [==============================] - 29s 1ms/step - loss: 0.3738 - acc: 0.8349 - val_loss: 0.5036 - val_acc: 0.7528


In [16]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score with LSTM:', score)
print('Test accuracy with LSTM:', acc)

25000/25000 [==============================] - 9s 351us/step
Test score with LSTM: 0.4953634677314758
Test accuracy with LSTM: 0.76464


## See Pre-trained models

https://nlp.stanford.edu/projects/glove/

## See Embeddings projector

http://projector.tensorflow.org/